In [ ]:
from google.colab import drive
import shutil
import tarfile
import os

drive.mount('/content/drive')

# Copy the file from Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/egovlp_fp16.tar.gz'  # Replace with your file path in Google Drive

tar = tarfile.open(file_path, "r")
tar.extractall(path="/content/ego4d_data/v1/egovlp_fp16")
tar.close()

Mounted at /content/drive


In [ ]:
%%bash

git clone https://github.com/EGO4D/episodic-memory
cd episodic-memory
git pull
git checkout nlq_fixes_and_fp16_support

In [ ]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = "access_key"
os.environ['AWS_SECRET_ACCESS_KEY'] = "secret_key"

In [ ]:
# Download the AWS and Ego4D CLIs, then download the annotations locally
%%bash

# Set up the AWS CLI
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -o awscliv2.zip >/dev/null
sudo ./aws/install >/dev/null 2>&1
aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID" && aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY"
rm "awscliv2.zip"

In [ ]:
# Set up the Ego4D CLI
!pip install ego4d

In [ ]:
# Download the Ego4D Annotations to ego4d_data/
!ego4d --output_directory="/content/ego4d_data" --datasets annotations --benchmarks nlq --version v1 -y

In [ ]:
# Ensure we have downloaded the files correctly
!ls /content/ego4d_data/v1/annotations | grep nlq

In [ ]:
!ls /content/ego4d_data/v1/egovlp_fp16 | wc -l

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ann_data = json.load(open("/content/ego4d_data/v1/annotations/nlq_train.json"))

In [ ]:
# please see https://ego4d-data.org/docs/data/annotations-schemas/

anns = []
for vid in ann_data["videos"]:
    for clip in vid["clips"]:
        for ann in clip["annotations"]:
            for query in ann["language_queries"]:
                anns.append({
                    "query_start_time_sec": clip["video_start_sec"],
                    "query_end_time_sec": clip["video_end_sec"],
                    "query_response_start_time_sec": query["video_start_sec"],
                    "query_response_end_time_sec": query["video_end_sec"],
                    "query_template": query.get("template", None),
                    "query": query.get("query", None),
                })

In [ ]:
num_queries = len(anns)
relative_query_sizes = np.array([
    (a["query_response_end_time_sec"] - a["query_response_start_time_sec"]) / (a["query_end_time_sec"] - a["query_start_time_sec"])
    for a in anns
])
query_sizes = np.array([
    (a["query_response_end_time_sec"] - a["query_response_start_time_sec"])
    for a in anns
])
clip_sizes = np.array([
    (a["query_end_time_sec"] - a["query_start_time_sec"])
    for a in anns
])

In [ ]:
query_sizes.max(), query_sizes.min(), query_sizes.std()

In [ ]:
# less than or equal to 4 frames => 9% of training data
(query_sizes <= 4/30).sum() / len(relative_query_sizes)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 9)

In [ ]:
plt.hist(relative_query_sizes[relative_query_sizes < 0.2], density=True, bins=128)
plt.show()

In [ ]:
plt.hist(relative_query_sizes[relative_query_sizes > 0.2], density=True, bins=128)
plt.show()

In [ ]:
clip_sizes.mean(), clip_sizes.max(), clip_sizes.min(), clip_sizes.std(), np.median(clip_sizes)

In [ ]:
plt.hist(clip_sizes)
plt.show()

In [ ]:
with open("vars.sh", "w") as out_f:
  out_f.write("""
export NAME=egovlp_fp16
export TASK_NAME=nlq_official_v1_$NAME
export BASE_DIR=data/dataset/nlq_official_v1_$NAME
export FEATURE_BASE_DIR=data/features/nlq_official_v1_$NAME/
export FEATURE_DIR=$FEATURE_BASE_DIR/video_features
export MODEL_BASE_DIR=/content/nlq_official_v1/checkpoints/

cd episodic-memory/NLQ/VSLNet
"""
  )


In [ ]:
%%bash

source vars.sh

echo $FEATURE_BASE_DIR
mkdir -p $FEATURE_BASE_DIR
# ln -s /content/ego4d_data/v1/omnivore_video_swinl $FEATURE_DIR
ln -s /content/ego4d_data/v1/egovlp_fp16 $FEATURE_DIR

In [ ]:
%%bash
%%capture

source vars.sh
pip install nltk submitit torch torchaudio torchvision tqdm transformers tensorboard Pillow terminaltables

In [ ]:
%%bash

source vars.sh

python utils/prepare_ego4d_dataset.py \
    --input_train_split /content/ego4d_data/v1/annotations/nlq_train.json \
    --input_val_split /content/ego4d_data/v1/annotations/nlq_val.json \
    --input_test_split /content/ego4d_data/v1/annotations/nlq_test_unannotated.json \
    --video_feature_read_path /content/ego4d_data/v1/egovlp_fp16 \
    --clip_feature_save_path $FEATURE_BASE_DIR/official \
    --output_save_path $BASE_DIR

In [ ]:
%load_ext tensorboard

In [ ]:
!mkdir -p /content/episodic-memory/NLQ/VSLNet//runs/
%tensorboard --logdir /content/episodic-memory/NLQ/VSLNet/runs/

In [ ]:
%%bash

source vars.sh

# machine parameters
export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export VAL_JSON_PATH="/content/ego4d_data/v1/annotations/nlq_val.json"

# hyper parameters
export BATCH_SIZE=32
export DIM=128
export NUM_EPOCH=10
export MAX_POS_LEN=128
export INIT_LR=0.0025

export TB_LOG_NAME="${NAME}_bs${BATCH_SIZE}_dim${DIM}_epoch${NUM_EPOCH}_ilr${INIT_LR}"

python main.py \
    --task $TASK_NAME \
    --predictor bert \
    --dim $DIM \
    --mode train \
    --video_feature_dim 256 \
    --max_pos_len $MAX_POS_LEN \
    --init_lr $INIT_LR \
    --epochs $NUM_EPOCH \
    --batch_size $BATCH_SIZE \
    --fv official \
    --num_workers $NUM_WORKERS \
    --data_loader_workers $DATALOADER_WORKERS \
    --model_dir $MODEL_BASE_DIR/$NAME \
    --eval_gt_json $VAL_JSON_PATH \
    --log_to_tensorboard $TB_LOG_NAME \
    --tb_log_freq 5 \
    --remove_empty_queries_from train